In [1]:
!pip install wandb -qU

     |████████████████████████████████| 1.8 MB 9.5 MB/s 
     |████████████████████████████████| 181 kB 63.0 MB/s 
     |████████████████████████████████| 162 kB 45.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 162 kB 55.1 MB/s 
     |████████████████████████████████| 158 kB 48.6 MB/s 
     |████████████████████████████████| 157 kB 53.7 MB/s 
     |████████████████████████████████| 157 kB 55.7 MB/s 
     |████████████████████████████████| 157 kB 57.5 MB/s 
     |████████████████████████████████| 157 kB 55.2 MB/s 
     |████████████████████████████████| 157 kB 55.0 MB/s 
     |████████████████████████████████| 157 kB 66.7 MB/s 
     |████████████████████████████████| 157 kB 47.0 MB/s 
     |████████████████████████████████| 156 kB 53.7 MB/s 


In [3]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: Currently logged in as: elvinagam. Use `wandb login --relogin` to force relogin


True

In [4]:
import random

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="basic-intro", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })
  
  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset
      
      # 🐝 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})
      
  # Mark the run as finished
  wandb.finish()

acc,▁▃▇█▆▇▇▇
loss,█▆▄▂▂▁▂▁
acc,0.87161
loss,0.06075


acc,▁▆▇▇▇▇█▇
loss,██▄▄▂▂▂▁
acc,0.88732
loss,0.0529


acc,▁▅▆█▇█▇▇
loss,██▅▂▂▂▁▂
acc,0.78906
loss,0.19843


acc,▁▃▆▇▆█▇█
loss,█▄▂▃▁▂▁▁
acc,0.78068
loss,0.19881


acc,▁▄▇████▇
loss,█▆▆▁▃▁▁▂
acc,0.71713
loss,0.30682


# 🔥 Simple Pytorch Neural Network

💪 Run this model to train a simple MNIST classifier, and click on the project page link to see your results stream in live to a W&B project.


Any run in `wandb` automatically logs [metrics](https://docs.wandb.ai/ref/app/pages/run-page#charts-tab),
[system information](https://docs.wandb.ai/ref/app/pages/run-page#system-tab),
[hyperparameters](https://docs.wandb.ai/ref/app/pages/run-page#overview-tab),
[terminal output](https://docs.wandb.ai/ref/app/pages/run-page#logs-tab) and
you'll see an [interactive table](https://docs.wandb.ai/guides/data-vis)
with model inputs and outputs.

## Set up Dataloader

In [5]:
#@title
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset, 
                                         batch_size=batch_size, 
                                         shuffle=True if is_train else False, 
                                         pin_memory=True, num_workers=2)
    return loader

def get_model(dropout):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

## Train Your Model

In [6]:
# Launch 5 experiments, trying different dropout rates
for _ in range(5):
    # 🐝 initialise a wandb run
    wandb.init(
        project="pytorch-intro",
        config={
            "epochs": 10,
            "batch_size": 128,
            "lr": 1e-3,
            "dropout": random.uniform(0.01, 0.80),
            })
    
    # Copy your config 
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)
    
    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
            example_ct += len(images)
            metrics = {"train/train_loss": train_loss, 
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch, 
                       "train/example_ct": example_ct}
            
            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb 
                wandb.log(metrics)
                
            step_ct += 1

        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss, 
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})
        
        print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # 🐝 Close your wandb run 
    wandb.finish()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▅▄▂▂▂▂▃▂▃▂▂▂▂▁▂▁▂▁▁▁▁▁▁▂▁▁▂▁▂▁▁▁▂▁▁▁▁▁
val/val_accuracy,▁▃▅▆▇▇▇▆█▇
val/val_loss,█▅▄▃▂▂▁▂▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.04875
val/val_accuracy,0.951
val/val_loss,0.15308


Train Loss: 0.296, Valid Loss: 0.294478, Accuracy: 0.92
Train Loss: 0.242, Valid Loss: 0.236189, Accuracy: 0.93
Train Loss: 0.288, Valid Loss: 0.205883, Accuracy: 0.94
Train Loss: 0.121, Valid Loss: 0.191412, Accuracy: 0.94
Train Loss: 0.165, Valid Loss: 0.181639, Accuracy: 0.94
Train Loss: 0.173, Valid Loss: 0.175348, Accuracy: 0.95
Train Loss: 0.060, Valid Loss: 0.164139, Accuracy: 0.95
Train Loss: 0.211, Valid Loss: 0.156899, Accuracy: 0.95
Train Loss: 0.051, Valid Loss: 0.155475, Accuracy: 0.95
Train Loss: 0.158, Valid Loss: 0.151235, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▄▄▃▂▃▃▄▂▃▂▂▂▁▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
val/val_accuracy,▁▃▆▆▆▇█▇██
val/val_loss,█▅▄▃▂▂▂▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.15839
val/val_accuracy,0.9515
val/val_loss,0.15123


Train Loss: 0.320, Valid Loss: 0.306857, Accuracy: 0.91
Train Loss: 0.479, Valid Loss: 0.246000, Accuracy: 0.92
Train Loss: 0.380, Valid Loss: 0.217085, Accuracy: 0.94
Train Loss: 0.166, Valid Loss: 0.207641, Accuracy: 0.94
Train Loss: 0.190, Valid Loss: 0.193019, Accuracy: 0.94
Train Loss: 0.289, Valid Loss: 0.182527, Accuracy: 0.94
Train Loss: 0.195, Valid Loss: 0.176133, Accuracy: 0.94
Train Loss: 0.098, Valid Loss: 0.173057, Accuracy: 0.94
Train Loss: 0.220, Valid Loss: 0.171810, Accuracy: 0.95
Train Loss: 0.089, Valid Loss: 0.163475, Accuracy: 0.95


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,█▅▃▃▃▃▂▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▂▁▂▁
val/val_accuracy,▁▃▅▆▆▆▇▇██
val/val_loss,█▅▄▃▂▂▂▁▁▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,120000
train/train_loss,0.089
val/val_accuracy,0.949
val/val_loss,0.16347


# 🔔 Try W&B Alerts

**[W&B Alerts](https://docs.wandb.ai/guides/track/alert)** allows you to send alerts, triggered from your Python code, to your Slack or email. There are 2 steps to follow the first time you'd like to send a Slack or email alert, triggered from your code:

1) Turn on Alerts in your W&B [User Settings](https://wandb.ai/settings)

2) Add `wandb.alert()` to your code:

```python
wandb.alert(
    title="Low accuracy", 
    text=f"Accuracy is below the acceptable threshold"
)
```

See the minimal example below to see how to use `wandb.alert`. You can find the full docs for **[W&B Alerts here](https://docs.wandb.ai/guides/track/alert)**

In [7]:
# Start a wandb run
wandb.init(project="pytorch-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f'Accuracy is: {accuracy}, {acc_threshold}')
    
    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title='Low Accuracy',
            text=f'Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}',
        )
        print('Alert triggered')
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 0.791, 0.3
Accuracy is: 1.386, 0.3
Accuracy is: 1.238, 0.3
Accuracy is: 0.46, 0.3
Accuracy is: 1.354, 0.3
Accuracy is: 0.727, 0.3
Accuracy is: 1.489, 0.3
Accuracy is: 1.511, 0.3
Accuracy is: 1.658, 0.3
Accuracy is: 1.007, 0.3
Accuracy is: 1.598, 0.3
Accuracy is: 1.74, 0.3
Accuracy is: 1.141, 0.3
Accuracy is: 1.273, 0.3
Accuracy is: 0.833, 0.3
Accuracy is: 1.407, 0.3
Accuracy is: 0.928, 0.3
Accuracy is: 0.32, 0.3
Accuracy is: 1.089, 0.3
Accuracy is: 1.415, 0.3
Accuracy is: 0.688, 0.3
Accuracy is: 0.917, 0.3
Accuracy is: 0.802, 0.3
Accuracy is: 1.144, 0.3
Accuracy is: 0.845, 0.3
Accuracy is: 1.247, 0.3
Accuracy is: 0.602, 0.3
Accuracy is: 0.78, 0.3
Accuracy is: 0.758, 0.3
Accuracy is: 1.027, 0.3
Accuracy is: 0.633, 0.3
Accuracy is: 0.055, 0.3
Alert triggered


Accuracy,▄▇▆▃▆▄▇▇█▅▇█▆▆▄▇▅▂▅▇▄▅▄▆▄▆▃▄▄▅▃▁
Accuracy,0.055
